In [191]:
import matplotlib.pyplot as plt
import pandas as pd

from io import StringIO
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [192]:
df = pd.read_csv('credit_score_classification.csv', low_memory=False)

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [197]:
df.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


# Ajuste de data types

In [196]:
df.loc[:, 'Outstanding_Debt':'Monthly_Balance'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Outstanding_Debt          100000 non-null  object  
 1   Credit_Utilization_Ratio  100000 non-null  float64 
 2   Credit_History_Age        90970 non-null   object  
 3   Payment_of_Min_Amount     100000 non-null  object  
 4   Total_EMI_per_month       100000 non-null  float64 
 5   Amount_invested_monthly   95521 non-null   object  
 6   Payment_Behaviour         100000 non-null  category
 7   Monthly_Balance           98800 non-null   object  
dtypes: category(1), float64(2), object(5)
memory usage: 5.4+ MB


In [ ]:
df['ID'] = df['ID'].astype('string')
df['Customer_ID'] = df['Customer_ID'].astype('string')
df['Month'] = df['Month'].astype('string')
df['Name'] = df['Name'].astype('string')
df['Age'] = df['Age'].str.replace('_', '').astype('int')
df['SSN'] = df['SSN'].astype('string')
df['Occupation'] = df['Occupation'].astype('string')
df['Annual_Income'] = df['Annual_Income'].str.replace('_', '').astype(float)
df['Num_of_Loan'] = df['Num_of_Loan'].str.replace('_', '').astype(int)
df['Type_of_Loan'] = df['Type_of_Loan'].astype('string')
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.replace('_', '').astype('float').fillna(0).astype('int')
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].str.replace('_', '0').astype('float').fillna(0)
df['Credit_Mix'] = df['Credit_Mix'].str.replace('_', 'Não informado').astype('string')
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace('_', '').astype('float')
df['Credit_History_Age'] = df['Credit_History_Age'].fillna('Não informado').astype('string')
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].astype('string')
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].str.replace('_', '').astype(float)
df['Monthly_Balance'] = df['Monthly_Balance'].str.replace('_', '').astype(float)
df['Credit_Score'] = df['Credit_Score'].astype('category')

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   ID                        100000 non-null  string  
 1   Customer_ID               100000 non-null  string  
 2   Month                     100000 non-null  string  
 3   Name                      90015 non-null   string  
 4   Age                       100000 non-null  int64   
 5   SSN                       100000 non-null  string  
 6   Occupation                100000 non-null  string  
 7   Annual_Income             100000 non-null  float64 
 8   Monthly_Inhand_Salary     84998 non-null   float64 
 9   Num_Bank_Accounts         100000 non-null  int64   
 10  Num_Credit_Card           100000 non-null  int64   
 11  Interest_Rate             100000 non-null  int64   
 12  Num_of_Loan               100000 non-null  int64   
 13  Type_of_Loan              8859